# Quick start: SATRAP CTI analysis toolbox

Import the SATRAP CTI analysis toolbox and some formatting utils.

In [ ]:
from satrap.service.satrap_analysis import CTIanalysisToolbox
from satrap.commons.format_utils import tabulate_groups, format_dict, tabulate_stix_obj
from satrap.settings import TYPEDB_SERVER_ADDRESS, DB_NAME

satrap = CTIanalysisToolbox(TYPEDB_SERVER_ADDRESS, DB_NAME)


## Starting with simple functions

Get the number of STIX Domain Objects (SDOs) in the knowledge base per STIX type.

In [ ]:
print(satrap.get_sdo_stats())

Get information about a STIX object using its STIX id.

In [ ]:
try:
	sdo = satrap.search_by_stix_id("campaign--0c259854-4044-4f6c-ac49-118d484b3e3b")
	print(tabulate_stix_obj(sdo))
except ValueError as e:
	print(e)

Search STIX objects by name or alias

In [ ]:
try:
	print(*satrap.search_by_alias_name("apt"),sep="\n")
except Exception as e:
	print(e)

Obtain the MITRE ATT&CK id of an element with its STIX id.

In [ ]:
print(satrap.get_mitre_id("attack-pattern--2aed01ad-3df3-4410-a8cb-11ea4ded587c"))

Get information on a specific MITRE ATT&CK element (technique, group, software, etc.) using its MITRE ATT&CK id.

In [ ]:
try:
	t1 = satrap.search_by_mitre_id("T1027.001")
	for data in t1:
		print(tabulate_stix_obj(data))
except ValueError as exc:
	print(exc)

Print the catalog of MITRE ATT&CK techniques and subtechniques. Subtechniques can be optionally filtered out.

In [ ]:
# print(satrap.mitre_attack_techniques(subtechniques=False))
print(satrap.mitre_attack_techniques())


Print the MITRE ATT&CK catalog of mitigations.

In [ ]:
print(satrap.mitre_attack_mitigations())

Retrieve mitigations explicitly associated to a specific technique using its STIX id.

In [ ]:
mitigations = satrap.mitigations_for_technique("attack-pattern--d1fcf083-a721-4223-aedf-bf8960798d62")
display(mitigations)

Get all MITRE ATT&CK groups, optionally filtering by keywords contained in their descriptions (case insensitive).

In [ ]:
display(satrap.mitre_attack_groups(["energy", "latin america"]))

Choose whether to get results including all the keywords (default) or any of them.

In [ ]:
groups = satrap.mitre_attack_groups(["energy", "latin america"], all=False)
print(f"{len(groups)} groups found")
print(tabulate_groups(groups))


## CTI analysis through automated reasoning

The following functions use the inference engine to draw relations on the data explicitly stored in the knowledge base. For some of them, the use of inference is optional.

Get statistics on the usage of ATT&CK techniques by groups. The output of this function is the same as when running the command `satrap techniques` on the CLI.

In [ ]:
# Run without the use of the inference engine
try:
	print("( MITRE ATT&CK technique, Name, Used by (num. intrusion sets) )")
	display(satrap.summarize_techniques_usage())
except Exception as err:
	print(err)

In [ ]:
# Run using the inference engine
try:
	print("( MITRE ATT&CK technique, Name, Used by (num. intrusion sets) )")
	display(satrap.summarize_techniques_usage(infer=True))
except Exception as err:
	print(err)

Get the techniques used by a specific group. Optionally enable inference.

In [ ]:
techniques = satrap.techniques_used_by_groups(["G0025"], infer=True)
display(techniques)

Look further into techniques used by all the groups in a list. Inference is optional.

In [ ]:
group_ids = ['G0073', 'G1015', 'G1004']

# without inference enabled
intersect = satrap.get_techniques_used_by_all(group_ids)
print(f"{len(intersect)} techniques used by groups {group_ids}")
print(format_dict(intersect))

In [ ]:
# using the inference engine
intersect = satrap.get_techniques_used_by_all(group_ids, True)
print(f"{len(intersect)} techniques used by groups {group_ids}")
print(format_dict(intersect))

Obtain the set of courses of action found to mitigate any of the techniques used by a specific group.

In [ ]:
display(satrap.related_mitigations(group_name="BlackTech"))

### Explanation of inferred knowledge

_Explain_ functions output `InferredAnswer` objects. An inferred answer consists of:
- The underlying query run by the function.
- A list of explanations showing why each element of the result set has been included.

An *explanation* consists of five elements:
- **Data inferred for query statement Q:** the statement in the query that triggered the application of an inference rule  
- **Applied rule:** the inference rule that was applied  
- **Condition met:** shows how the rule's conditions are satisfied by data in the CTI SKB.  
- **Inferred conclusion:** is the result of the application of the rule.  
- **Mapping of variables:** shows how the variables in the query statement Q map to the variables of the rule applied.


**Hint:** For an interactive visual explanation, we suggest to copy the query returned by an explain function in TypeDB Studio and run it there as explained in the user manual (at `docs/manual/interfaces.md`).

Obtain an explanation for why (and if) a given mitigation is found to address the techniques used by a group. This function can be used to explain individual courses of action in the output of `related_mitigations` shown above.

For instance, to know why "Remote Data Storage" is found to be a course of action against "BlackTech":

In [ ]:
try:
	rel_explanation = satrap.explain_if_related_mitigation("G0098", "course-of-action--20a2baeb-98c2-4901-bad7-dc62d0a03dea")
	print(f"EXECUTED QUERY:\n{rel_explanation.query}\n")
	print("EXPLANATIONS:")
	for tech in rel_explanation.explanations:
		print(format_dict(tech.as_json()), end="\n")
except ValueError as err:
	print(err)

Obtain relevant courses of action for a given intrusion set, when considering the mitigation of a specific SDO (usually technique).

E.g., What mitigations can we find against "ZIRCONIUM" that specifically address the technique "T1059.006"?

In [ ]:
try:
	reason = satrap.explain_related_techniques("ZIRCONIUM", "T1059.006")
	print(f"EXECUTED QUERY\n{reason.query}\n")
	for tech in reason.explanations:
		print(format_dict(tech.as_json()), end="\n")
except Exception as exc:
	print(exc)

Find out, with an explanation, the set of techniques used by a specific group. Optionally filter for the explanation of a specific technique.

In [ ]:
try:
	dg_explanation = satrap.explain_techniques_used_by_groups(["G1015"], "T1069.002")
	for tech in dg_explanation.explanations:
		print(format_dict(tech.as_json()), end="\n")
except Exception as err:
	print(err)